In [3]:


import warnings

from abc import ABCMeta, abstractmethod


import numpy as np

import seaborn as sn
import matplotlib.pyplot as plt
import pandas
from sklearn.metrics import accuracy_score
from sklearn import naive_bayes as nb
import numpy as np
from sklearn.model_selection import train_test_split


from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import binarize
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import label_binarize
from sklearn.utils import check_X_y, check_array, deprecated
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.utils.fixes import logsumexp
from sklearn.utils.multiclass import _check_partial_fit_first_call
from sklearn.utils.validation import check_is_fitted, check_non_negative, column_or_1d
from sklearn.utils.validation import _check_sample_weight

__all__ = ['BernoulliNB', 'GaussianNB', 'MultinomialNB', 'ComplementNB',
           'CategoricalNB']


class _BaseNB(ClassifierMixin, BaseEstimator, metaclass=ABCMeta):
    """Abstract base class for naive Bayes estimators"""

    @abstractmethod
    def _joint_log_likelihood(self, X):
        """Compute the unnormalized posterior log probability of X

        I.e. ``log P(c) + log P(x|c)`` for all rows x of X, as an array-like of
        shape [n_classes, n_samples].

        Input is passed to _joint_log_likelihood as-is by predict,
        predict_proba and predict_log_proba.
        """

    def _check_X(self, X):
        """To be overridden in subclasses with the actual checks."""
        # Note that this is not marked @abstractmethod as long as the
        # deprecated public alias sklearn.naive_bayes.BayesNB exists
        # (until 0.24) to preserve backward compat for 3rd party projects
        # with existing derived classes.
        return X

    def predict(self, X):
        """
        Perform classification on an array of test vectors X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        C : ndarray of shape (n_samples,)
            Predicted target values for X
        """
        check_is_fitted(self)
        X = self._check_X(X)
        jll = self._joint_log_likelihood(X)
        return self.classes_[np.argmax(jll, axis=1)]

    def predict_log_proba(self, X):
        """
        Return log-probability estimates for the test vector X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        C : array-like of shape (n_samples, n_classes)
            Returns the log-probability of the samples for each class in
            the model. The columns correspond to the classes in sorted
            order, as they appear in the attribute :term:`classes_`.
        """
        check_is_fitted(self)
        X = self._check_X(X)
        jll = self._joint_log_likelihood(X)
        # normalize by P(x) = P(f_1, ..., f_n)
        log_prob_x = logsumexp(jll, axis=1)
        return jll - np.atleast_2d(log_prob_x).T

    def predict_proba(self, X):
        """
        Return probability estimates for the test vector X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)

        Returns
        -------
        C : array-like of shape (n_samples, n_classes)
            Returns the probability of the samples for each class in
            the model. The columns correspond to the classes in sorted
            order, as they appear in the attribute :term:`classes_`.
        """
        return np.exp(self.predict_log_proba(X))


class GaussianNB(_BaseNB):


    def __init__(self, priors=None, var_smoothing=1e-9):
        self.priors = priors
        print("asjhask")
        print(priors)
        self.var_smoothing = var_smoothing
        print(var_smoothing)

    def fit(self, X, y, sample_weight=None):

        y = column_or_1d(y, warn=True)
        print("--------y--------------")
        print(y)
        return self._partial_fit(X, y, np.unique(y), _refit=True,
                                 sample_weight=sample_weight)

    def _check_X(self, X):
        return check_array(X)

    @staticmethod
    def _update_mean_variance(n_past, mu, var, X, sample_weight=None):
        
        if X.shape[0] == 0:
          return mu, var

        # Compute (potentially weighted) mean and variance of new datapoints
        if sample_weight is not None:
            n_new = float(sample_weight.sum())
            new_mu = np.average(X, axis=0, weights=sample_weight)
            new_var = np.average((X - new_mu) ** 2, axis=0,
                                 weights=sample_weight)
        else:
            n_new = X.shape[0]
            new_var = np.var(X, axis=0)
            new_mu = np.mean(X, axis=0)

            print("------------var+mu---------------")

            print(new_var)
            print(new_mu)


        if n_past == 0:
            print("wow")
            return new_mu, new_var

        n_total = float(n_past + n_new)

        # Combine mean of old and new data, taking into consideration
        # (weighted) number of observations
        total_mu = (n_new * new_mu + n_past * mu) / n_total
        # Combine variance of old and new data, taking into consideration
        # (weighted) number of observations. This is achieved by combining
        # the sum-of-squared-differences (ssd)
        old_ssd = n_past * var
        new_ssd = n_new * new_var
        total_ssd = (old_ssd + new_ssd +
                     (n_new * n_past / n_total) * (mu - new_mu) ** 2)
        total_var = total_ssd / n_total

        return total_mu, total_var

    def partial_fit(self, X, y, classes=None, sample_weight=None):
       
        
        return self._partial_fit(X, y, classes, _refit=False,
                                 sample_weight=sample_weight)

    def _partial_fit(self, X, y, classes=None, _refit=False,
                     sample_weight=None):
       
        print("Actual fitting GNB")
        X, y = check_X_y(X, y)
        print(X)
        print(y)
        if sample_weight is not None:
            sample_weight = _check_sample_weight(sample_weight, X)

        self.epsilon_ = self.var_smoothing * np.var(X, axis=0).max()
        print("Selfepsilom")
        print( np.var(X, axis=0))
        print(self.epsilon_)

        if _refit:
            self.classes_ = None

        if _check_partial_fit_first_call(self, classes):
            # This is the first call to partial_fit:
            # initialize various cumulative counters
            n_features = X.shape[1]
            n_classes = len(self.classes_)
            self.theta_ = np.zeros((n_classes, n_features))
            self.sigma_ = np.zeros((n_classes, n_features))
            

            self.class_count_ = np.zeros(n_classes, dtype=np.float64)

            # Initialise the class prior
            # Take into account the priors
            if self.priors is not None:
                priors = np.asarray(self.priors)
                print(priors)
                # Check that the provide prior match the number of classes
                if len(priors) != n_classes:
                    raise ValueError('Number of priors must match number of'
                                     ' classes.')
                # Check that the sum is 1
                if not np.isclose(priors.sum(), 1.0):
                    raise ValueError('The sum of the priors should be 1.')
                # Check that the prior are non-negative
                if (priors < 0).any():
                    raise ValueError('Priors must be non-negative.')
                self.class_prior_ = priors
            else:
                # Initialize the priors to zeros for each class
                self.class_prior_ = np.zeros(len(self.classes_),
                                             dtype=np.float64)
        
        else:
            if X.shape[1] != self.theta_.shape[1]:
                msg = "Number of features %d does not match previous data %d."
                raise ValueError(msg % (X.shape[1], self.theta_.shape[1]))
            # Put epsilon back in each time
            self.sigma_[:, :] -= self.epsilon_

        classes = self.classes_

        unique_y = np.unique(y)
        
        unique_y_in_classes = np.in1d(unique_y, classes)
        

        if not np.all(unique_y_in_classes):
            raise ValueError("The target label(s) %s in y do not exist in the "
                             "initial classes %s" %
                             (unique_y[~unique_y_in_classes], classes))

        for y_i in unique_y:
            i = classes.searchsorted(y_i)
            X_i = X[y == y_i, :]
            print(str(y_i)+" for the label")
            print(X_i)

            if sample_weight is not None:
                sw_i = sample_weight[y == y_i]
                N_i = sw_i.sum()
            else:
                sw_i = None
                N_i = X_i.shape[0]
                print(N_i)

            new_theta, new_sigma = self._update_mean_variance(
                self.class_count_[i], self.theta_[i, :], self.sigma_[i, :],
                X_i, sw_i)

            self.theta_[i, :] = new_theta
            self.sigma_[i, :] = new_sigma
            self.class_count_[i] += N_i
            print(self.class_count_[i])
            print("---------self theta-------------")
            print(self.theta_)
            print("---------self sigma-------------")
            print(self.sigma_)

        self.sigma_[:, :] += self.epsilon_

        # Update if only no priors is provided
        if self.priors is None:
            # Empirical prior, with sample_weight taken into account
            self.class_prior_ = self.class_count_ / self.class_count_.sum()
            print(self.class_count_)
            print(self.class_prior_)
            

        return self

    def _joint_log_likelihood(self, X):
        joint_log_likelihood = []
        for i in range(np.size(self.classes_)):
            jointi = np.log(self.class_prior_[i])
            print("Selffffffsigma")
            
            print(self.sigma_[i,:])
            
            print("n_ij_first")
            n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
            print(n_ij)
            print("n_ij_final")
            n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) /
                                 (self.sigma_[i, :]), axis = 1)
            print(n_ij)
            
            joint_log_likelihood.append(jointi + n_ij)

        joint_log_likelihood = np.array(joint_log_likelihood).T
        print("Joint_log_liklihood")
        print(joint_log_likelihood)
        return joint_log_likelihood





dataframe = pandas.read_table('/content/dos200_labels.txt')
data = dataframe.drop("No.", axis=1)

dataset = data.values



# split into input (X) and output (Y) variables
training = 25825
testing = 36893 - 25825

l = 36893

X = dataset[:,0:]
# print(X.shape)
labels = []
for i in range (0,l):
  if X[i][9] == 1:
    labels.append(1)
  else:
    labels.append(0)

features = X[0:l,0:9]




# Split our data
train, test, train_labels, test_labels = features[0:training],features[training:testing+training],labels[0:training],labels[training:testing+training]

  # Initialize our classifier



gnb = GaussianNB()




# Train our classifier
print(len(train_labels))
model =gnb.fit(train,train_labels)
preds = gnb.predict(test)
print("----------------------------GNB--------------------------")
print(accuracy_score(test_labels, preds))


asjhask
None
1e-09
25825
--------y--------------
[1 1 1 ... 1 1 1]
Actual fitting GNB
[[2.40000000e+01 3.80000000e+01 4.00000000e+00 ... 4.18842832e-02
  8.42430489e-02 1.45552347e-02]
 [2.40000000e+01 3.70000000e+01 4.00000000e+00 ... 4.42486976e-02
  7.68504328e-02 1.52779076e-02]
 [2.50000000e+01 3.50000000e+01 3.00000000e+00 ... 4.22065228e-02
  8.61096838e-02 2.37975825e-02]
 ...
 [2.70000000e+01 4.10000000e+01 3.00000000e+00 ... 4.12082235e-02
  6.22576715e-02 1.67768202e-02]
 [2.40000000e+01 3.40000000e+01 3.00000000e+00 ... 4.19560708e-02
  8.43522175e-02 1.39475689e-02]
 [2.50000000e+01 4.90000000e+01 4.00000000e+00 ... 4.10179863e-02
  6.61039282e-02 1.52374725e-02]]
[1 1 1 ... 1 1 1]
Selfepsilom
[1.43927821e+00 7.44297638e+01 4.22075860e+01 2.32279032e-04
 4.24778891e+01 3.05271632e-03 4.44918970e-05 1.32300762e-02
 1.04442028e-05]
7.442976381538934e-08
0 for the label
[[2.50000000e+01 5.20000000e+01 2.50000000e+01 ... 2.14422587e-02
  4.54174424e-01 2.14422587e-02]
 [2.1000

In [ ]:
import warnings

from abc import ABCMeta, abstractmethod


import numpy as np

import seaborn as sn
import matplotlib.pyplot as plt
import pandas
from sklearn.metrics import accuracy_score
from sklearn import naive_bayes as nb
import numpy as np
from sklearn.model_selection import train_test_split

dataframe = pandas.read_table('/content/mixed200.txt')
data = dataframe.drop("No.", axis=1)
data = data.drop("StdPageRank", axis=1)
data = data.drop("VarPageRank", axis=1)
dataset = data.values



# split into input (X) and output (Y) variables
training = 57748
testing = 24749

l = 82497
X = dataset[:,0:]
# print(X.shape)
labels = []
for i in range (0,l):
  if X[i][9] == 1:
    labels.append(1)
  else:
    labels.append(0)

features = X[0:l,0:9]

print(labels.count(0))
print(labels.count(1))


# Split our data
train, test, train_labels, test_labels = features[0:training],features[training:testing+training],labels[0:training],labels[training:testing+training]




gnb = svm.SVC()




# Train our classifier
print(len(train_labels))
model =gnb.fit(train,train_labels)
preds = gnb.predict(test)
print("----------------------------SVM--------------------------")
print(accuracy_score(test_labels, preds))


23663
58834


NameError: ignored